In [8]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas_datareader as pdr
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import warnings


In [2]:
def backtest(ticker,initial_date,end_date):
    ticker=ticker
    qty= 2
    stock = yf.Ticker(ticker)
    initial_date=initial_date
    date_s = datetime.strptime(initial_date, '%Y-%m-%d')
    end_date=end_date
    date_e = datetime.strptime(end_date, '%Y-%m-%d')
    hist = stock.history(start=date_s, end=date_e)
    df=hist.drop(['Dividends','Stock Splits','Volume'],axis=1)
    ## 14_Day RSI
    df['Up Move'] = np.nan
    df['Down Move'] = np.nan
    df['Average Up'] = np.nan
    df['Average Down'] = np.nan
    # Relative Strength
    df['RS'] = np.nan
    # Relative Strength Index
    df['RSI'] = np.nan
    ## Calculate Up Move & Down Move
    for x in range(1, len(df)):
        df['Up Move'][x] = 0
        df['Down Move'][x] = 0

        if df['Close'][x] > df['Close'][x-1]:
            df['Up Move'][x] = df['Close'][x] - df['Close'][x-1]

        if df['Close'][x] < df['Close'][x-1]:
            df['Down Move'][x] = abs(df['Close'][x] - df['Close'][x-1])  

    ## Calculate initial Average Up & Down, RS and RSI
    df['Average Up'][14] = df['Up Move'][1:15].mean()
    df['Average Down'][14] = df['Down Move'][1:15].mean()
    df['RS'][14] = df['Average Up'][14] / df['Average Down'][14]
    df['RSI'][14] = 100 - (100/(1+df['RS'][14]))
    ## Calculate rest of Average Up, Average Down, RS, RSI
    for x in range(15, len(df)):
        df['Average Up'][x] = (df['Average Up'][x-1]*13+df['Up Move'][x])/14
        df['Average Down'][x] = (df['Average Down'][x-1]*13+df['Down Move'][x])/14
        df['RS'][x] = df['Average Up'][x] / df['Average Down'][x]
        df['RSI'][x] = 100 - (100/(1+df['RS'][x]))
    df['Signal']=0
    df['Take Profit']=0
    df['Sell']=0
    df['Buy']=0
    df1=df.drop(df.index[:14])
    stdv=df1["Open"].std()
    highest=df1["High"].max()
    take_profit_high=highest-stdv*.25
    take_profit_low=highest-stdv*.5
    df1["52weekhigh"] = df1['Close'].rolling(window =200).max()
    for index, row in df1.iterrows():
        if row["RSI"] < 28:
            df1.loc[index, "Signal"]= 1   
        if row["RSI"] > 70:
            df1.loc[index, "Signal"]= 2
        if row["Open"] > row['52weekhigh']-(stdv*.5) and row['52weekhigh']-(stdv*.25) > row["Open"]:
            df1.loc[index, "Signal"]= -1
    for index, row in df1.iterrows():
        if row["Signal"] == -1:    
            df1.loc[index, "Take Profit"]= row["Open"]
        if row["Signal"] == 1:    
            df1.loc[index, "Buy"]= row["Open"]
        if row["Signal"] == 2:    
            df1.loc[index, "Sell"]= row["Open"]
    df1['Signal'].value_counts()
    #df1['Open'].plot(figsize=(16, 9),color="orange")
    #df1['Take Profit'].plot(figsize=(16, 9),color="blue")
    #df1['Buy'].plot(figsize=(16, 9),color="yellow")
    #df1['Sell'].plot(figsize=(16, 9),color="gray")
    results=df1[df1['Signal']!=0]
    results['Excec_Sell']=0
    results['Excec_Buy']=0
    prev_signal=results['Signal'].shift()
    results.insert(11,'Prev Signal',prev_signal)
    for index, row in results.iterrows():
        if row["Signal"] == 2 and row["Prev Signal"] == -1:    
            results.loc[index, "Excec_Sell"]= row["Close"]
        if row["Signal"] == 1 and row["Prev Signal"] == -1:    
            results.loc[index, "Excec_Buy"]= row["Close"]
        if row["Signal"] == 2 and row["Prev Signal"] == 1:    
            results.loc[index, "Excec_Buy"]= row["Close"]
        if row["Signal"] == 1 and row["Prev Signal"] == 2:    
            results.loc[index, "Excec_Buy"]= row["Close"]
        if row["Signal"] == 1 and row["Prev Signal"] == 1:    
            results.loc[index, "Excec_Buy"]= row["Close"]
    excecutions=results[(results['Excec_Sell']!=0) | (results['Excec_Buy']!=0)]
    excecutions['AddSubs']=0
    for index, row in excecutions.iterrows():
        if row["Signal"] == 2:    
            excecutions.loc[index, "AddSubs"]= -1
        if row["Signal"] == 1:    
            excecutions.loc[index, "AddSubs"]= 1
    prev_addsub=excecutions['AddSubs'].shift()
    excecutions['AddSubbs_Prev']=prev_addsub
    prev_addsub1=excecutions['AddSubbs_Prev'].shift()
    excecutions['AddSubbs_Prev1']=prev_addsub1
    prev_addsub2=excecutions['AddSubbs_Prev1'].shift()
    excecutions['AddSubbs_Prev2']=prev_addsub2
    line1=df.iloc[14]
    line1=pd.DataFrame(line1)
    line2=line1.swapaxes("index", "columns")
    excecutions1 = pd.concat([line2, excecutions]).reset_index(drop = True)
    excecutions1['Signal'][0]=1
    open1=excecutions1.iloc[0,3]
    excecutions1['Excec_Buy'][0]=open1
    excecutions1['AddSubs'][0]=1
    excecutions1['AddSubbs_Prev'][0]=0
    excecutions1['AddSubbs_Prev'][1]=1
    excecutions1['AddSubbs_Prev1'][0]=0
    excecutions1['AddSubbs_Prev1'][1]=0
    excecutions1['AddSubbs_Prev1'][2]=1
    excecutions1['AddSubbs_Prev2'][0]=0
    excecutions1['AddSubbs_Prev2'][1]=0
    excecutions1['AddSubbs_Prev2'][2]=0
    excecutions1['AddSubbs_Prev2'][3]=1
    final=excecutions1.fillna(0)
    final.insert(21,'Prev_Acum',0)
    final=final.fillna(0)
    for index, row in final.iterrows():
        if row['AddSubs'] == 1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == -3:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == -2:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == -1:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == 0:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and row['AddSubbs_Prev'] == 1 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and row['AddSubbs_Prev'] == 1 and row['AddSubbs_Prev1'] == -1 and row['AddSubbs_Prev2'] == 1:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == 1 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=1,
        elif row['AddSubs'] == 1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == 1:
            final.loc[index,'Prev_Acum']=1,  
        elif row['AddSubs'] == 1 and row['AddSubbs_Prev'] == -1 and row['AddSubbs_Prev1'] == 1 and row['AddSubbs_Prev2'] == 1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == 1 and row['AddSubbs_Prev'] == 1 and row['AddSubbs_Prev1'] == 1 and row['AddSubbs_Prev2'] == -1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == 1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == 2:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == 1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == 3:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == -3:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == -2:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 1 and row['AddSubbs_Prev1'] == -1 and row['AddSubbs_Prev2'] == -1:
            final.loc[index,'Prev_Acum']=-1,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == -1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == -1 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == -1 and row['AddSubbs_Prev1'] == -1 and row['AddSubbs_Prev2'] == 1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == -1 and row['AddSubbs_Prev1'] == 1 and row['AddSubbs_Prev2'] == -1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == -1 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 1 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == -1:
            final.loc[index,'Prev_Acum']=-1,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 1 and row['AddSubbs_Prev1'] == -1 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=-1,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == 1 and row['AddSubbs_Prev2'] == -1:
            final.loc[index,'Prev_Acum']=-1,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == 0 and row['AddSubbs_Prev1'] == -1 and row['AddSubbs_Prev2'] == 1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == -1 and row['AddSubbs_Prev1'] == 1 and row['AddSubbs_Prev2'] == 0:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and row['AddSubbs_Prev'] == -1 and row['AddSubbs_Prev1'] == 0 and row['AddSubbs_Prev2'] == 1:
            final.loc[index,'Prev_Acum']=0,
        elif row['AddSubs'] == -1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == 1:
            final.loc[index,'Prev_Acum']=-1,
        elif row['AddSubs'] == -1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == 2:
            final.loc[index,'Prev_Acum']=-1,
        elif row['AddSubs'] == -1 and (row['AddSubbs_Prev']+row['AddSubbs_Prev1']+row['AddSubbs_Prev2']) == 3:
            final.loc[index,'Prev_Acum']=-1,
        else:
            pass
    trade_summary=final[final['Prev_Acum']!=0]
    trade_summary=trade_summary[['Open','High','Low','Close','Prev_Acum']]
    trade_summary['Excecution_P']=trade_summary['Prev_Acum']*-trade_summary['Close']
    trade_summary
    position=trade_summary['Prev_Acum'].sum()
    last_price=df['Close'][-1]
    remainder=trade_summary['Excecution_P'].sum()
    if trade_summary['Prev_Acum'].sum() == 2 and trade_summary['Prev_Acum'].count()>=3 and trade_summary['Prev_Acum'].iat[-1]==1 and trade_summary['Prev_Acum'].iat[-2]==1 and trade_summary['Prev_Acum'].iat[-3]==-1:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec[:-2])
        fi=trade_summary['Excecution_P'].iat[-2]
        si=trade_summary['Excecution_P'].iat[-1]
        av_c=(fi+si)/2
        unr_g=last_p*2+av_c*2
        t_r=unr_g+gains
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(t_r)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 2 and trade_summary['Prev_Acum'].count()>=3 and trade_summary['Prev_Acum'].iat[-1]==1 and trade_summary['Prev_Acum'].iat[-2]==-1 and trade_summary['Prev_Acum'].iat[-3]==1:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec[:-1])-trade_summary['Excecution_P'].iat[-3]
        fi=trade_summary['Excecution_P'].iat[-3]
        si=trade_summary['Excecution_P'].iat[-1]
        av_c=(fi+si)/2
        unr_g=last_p*2+av_c*2
        t_r=unr_g+gains
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(t_r)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 2 and trade_summary['Prev_Acum'].count()==2:
        last_p=df['Close'].iat[-1]
        fi=trade_summary['Excecution_P'].iat[-2]
        si=trade_summary['Excecution_P'].iat[-1]
        av_c=(fi+si)/2
        unr_g=av_c*2+last_p*2
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: 0'),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(unr_g)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 1 and trade_summary['Prev_Acum'].count()==1:
        last_p=df['Close'].iat[-1]
        fi=trade_summary['Excecution_P'].iat[-1]
        unr_g=last_p+fi
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: 0'),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(unr_g)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 1 and trade_summary['Prev_Acum'].count()>=3 and trade_summary['Prev_Acum'].iat[-1]==-1 and trade_summary['Prev_Acum'].iat[-2]==1 and trade_summary['Prev_Acum'].iat[-3]==-1:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec)-trade_summary['Excecution_P'].iat[-2]
        si=trade_summary['Excecution_P'].iat[-2]
        unr_g=last_p+si
        t_r=unr_g+gains
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(t_r)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 1 and trade_summary['Prev_Acum'].count()>=3 and trade_summary['Prev_Acum'].iat[-1]==-1 and trade_summary['Prev_Acum'].iat[-2]==1 and trade_summary['Prev_Acum'].iat[-3]==1:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec)-trade_summary['Excecution_P'].iat[-2]
        si=trade_summary['Excecution_P'].iat[-2]
        unr_g=last_p+si
        t_r=unr_g+gains
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(t_r)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 1 and trade_summary['Prev_Acum'].count()>=3 and trade_summary['Prev_Acum'].iat[-1]==1 and trade_summary['Prev_Acum'].iat[-2]==-1 and trade_summary['Prev_Acum'].iat[-3]==1:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec)-trade_summary['Excecution_P'].iat[-1]
        si=trade_summary['Excecution_P'].iat[-1]
        unr_g=last_p+si
        t_r=unr_g+gains
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(t_r)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 1 and trade_summary['Prev_Acum'].count()>=3 and trade_summary['Prev_Acum'].iat[-1]==1 and trade_summary['Prev_Acum'].iat[-2]==-1 and trade_summary['Prev_Acum'].iat[-3]==-1:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec)-trade_summary['Excecution_P'].iat[-1]
        si=trade_summary['Excecution_P'].iat[-1]
        unr_g=last_p+si
        t_r=unr_g+gains
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: '+ str(unr_g)),
        print('Total Gains: '+ str(t_r)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    elif trade_summary['Prev_Acum'].sum() == 0:
        last_p=df['Close'].iat[-1]
        excec=list(trade_summary['Excecution_P'])
        gains=sum(excec)
        first_p=trade_summary['Close'][0]
        bnhg=last_p*2-first_p*2
        print(str(ticker)),
        print('Model Results: '),
        print('Realized Gains: '+ str(gains)),
        print('Unrealized Gains: 0'),
        print('Total Gains: '+ str(gains)),
        print('                               '),
        print('Buy and Hold: '),
        print('Total Returns ' + str(bnhg)),
    else:
        pass

In [6]:
tickers=['ADBE','V','IXJ','VTV', 'ACWI','XLF', 'SOXX','NVDA','ATVI','SPY','IVV','MSFT','CIBR','XBI','GOOGL','DIS','BOTZ','XT','QQQ','AMZN','AAPL', 'SQQQ']

In [12]:
for i in tickers:
    backtest(i,'2011-01-01','2022-12-31'),
    warnings.filterwarnings('ignore'),
    print()

ADBE
Model Results: 
Realized Gains: 453.0800533294678
Unrealized Gains: -399.54998779296875
Total Gains: 53.53006553649902
                               
Buy and Hold: 
Total Returns 605.0

V
Model Results: 
Realized Gains: 153.75348281860352
Unrealized Gains: 35.363494873046875
Total Gains: 189.1169776916504
                               
Buy and Hold: 
Total Returns 382.5671691894531

IXJ
Model Results: 
Realized Gains: 55.73623085021973
Unrealized Gains: 0
Total Gains: 55.73623085021973
                               
Buy and Hold: 
Total Returns 127.0173225402832

VTV
Model Results: 
Realized Gains: 131.0771141052246
Unrealized Gains: 16.885833740234375
Total Gains: 147.96294784545898
                               
Buy and Hold: 
Total Returns 200.15077209472656

ACWI
Model Results: 
Realized Gains: 61.17890930175781
Unrealized Gains: 12.307212829589844
Total Gains: 73.48612213134766
                               
Buy and Hold: 
Total Returns 95.79672241210938

XLF
Model Resul